In [ ]:
from src.datasets.kitti_dataset import KittiDataset

dataset_path = "dataset/"
sequence = "00"
image_instances_path = "pipeline/vfm-labelss/sam/00/"
gt_labels_path = "dataset/sequences/00/labels/"
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [ ]:
from src.services.preprocessing.common.config import ConfigDTO

config = ConfigDTO(
    **{
        "dataset": kitti,
        "start_index": 20,
        "end_index": 24,
        "start_image_index_offset": 3,
        "cam_name": "cam2",
        "R": 18,
        "nb_neighbors": 25,
        "std_ratio": 5.0,
        "voxel_size": 0.25,
    }
)

In [ ]:
from src.services.preprocessing.init.map import InitMapProcessor
from src.services.preprocessing.init.instances_matrix import InitInstancesMatrixProcessor

init_pcd = InitMapProcessor().process(config)
points2instances = InitInstancesMatrixProcessor().process(config, init_pcd)

In [ ]:
from src.utils.gt_utils import build_sem_inst_label_arrays

sem_label_array_src, inst_label_array_src = build_sem_inst_label_arrays(gt_labels_path,
                                                                        config.start_index,
                                                                        config.end_index)
print(len(sem_label_array_src) == len(init_pcd.points))
print(len(inst_label_array_src) == len(init_pcd.points))

In [ ]:
points2instances.shape

In [ ]:
import copy

from src.utils.pcd_utils import color_pcd_by_labels
from src.utils.pcd_utils import visualize_pcd

colored_pcd = color_pcd_by_labels(copy.deepcopy(init_pcd), points2instances[:, 0])
visualize_pcd(colored_pcd)

In [ ]:
from src.services.preprocessing.not_zero import SelectionNotZeroProcessor
from src.services.preprocessing.in_cube import SelectionInCubeProcessor
from src.services.preprocessing.statistical_outlier import StatisticalOutlierProcessor

processors = [
    SelectionNotZeroProcessor(),
    SelectionInCubeProcessor(),
    StatisticalOutlierProcessor(),
]

In [ ]:
import copy

pcd = copy.deepcopy(init_pcd)
for processor in processors:
    pcd, points2instances, indices = processor.process(config, pcd, points2instances)
    sem_label_array_src = sem_label_array_src[indices]
    inst_label_array_src = inst_label_array_src[indices]

pcd_for_clustering = copy.deepcopy(pcd)
points2instances_pcd_for_clustering = copy.deepcopy(points2instances)
inst_label_array_for_clustering = copy.deepcopy(inst_label_array_src)
sem_label_array_for_clustering = copy.deepcopy(sem_label_array_src)

In [ ]:
len(inst_label_array_for_clustering) == len(pcd_for_clustering.points)

In [ ]:
len(sem_label_array_for_clustering) == len(pcd_for_clustering.points)

In [ ]:
from src.utils.gt_utils import combine_sem_inst_labels

combined_label_array = combine_sem_inst_labels(sem_label_array_for_clustering,
                                               inst_label_array_for_clustering)

In [ ]:
len(combined_label_array) == len(pcd_for_clustering.points)

In [ ]:
import copy

from src.utils.pcd_utils import color_pcd_by_labels
from src.utils.pcd_utils import visualize_pcd

colored_pcd_for_clustering = color_pcd_by_labels(copy.deepcopy(pcd_for_clustering),
                                                 points2instances_pcd_for_clustering[:, 3])
visualize_pcd(colored_pcd_for_clustering)

In [ ]:
colored_pcd_pcd_for_clustering = color_pcd_by_labels(copy.deepcopy(pcd_for_clustering), combined_label_array)
visualize_pcd(colored_pcd_pcd_for_clustering)

In [ ]:
from src.services.preprocessing.voxel_down import VoxelDownProcessor

pcd, points2instances, trace = VoxelDownProcessor().process(config, pcd, points2instances)

In [ ]:
colored_pcd_pcd_for_clustering = color_pcd_by_labels(copy.deepcopy(pcd_for_clustering), inst_label_array_for_clustering)
visualize_pcd(colored_pcd_pcd_for_clustering)

In [ ]:
import numpy as np

from scipy.spatial.distance import cdist
from src.utils.distances_utils import sam_label_distance

points = np.asarray(pcd.points)
spatial_distance = cdist(points, points)

dist, masks = sam_label_distance(points2instances, spatial_distance, 3, 3, 5)

In [ ]:
dist.shape

In [ ]:
from src.services.distance.isolated import RemovingIsolatedPointsProcessor
from src.services.distance.connected_component import ExtractionLargestConnectedComponentProcessor

distance_processors = [
    RemovingIsolatedPointsProcessor(),
    ExtractionLargestConnectedComponentProcessor(),
]

for processor in distance_processors:
    dist, points, trace = processor.process(dist, points, trace)

In [ ]:
print(dist.shape)
print(len(points))
print(len(trace))

In [ ]:
from src.services.normalized_cut_service import normalized_cut

T = 0.02
eigenval = 2

clusters = normalized_cut(
    dist,
    np.array([i for i in range(len(points))], dtype=int),
    T,
    eigenval
)

In [ ]:
len(clusters)

In [ ]:
from src.utils.pcd_utils import color_pcd_by_clusters_and_voxels

pcd_colored = color_pcd_by_clusters_and_voxels(pcd_for_clustering, trace, clusters)

In [ ]:
from src.utils.pcd_utils import visualize_pcd

visualize_pcd(pcd_colored)

In [ ]:
from src.utils.pcd_utils import color_pcd_by_labels

pcd_src_colored = color_pcd_by_labels(pcd_for_clustering, points2instances_pcd_for_clustering[:, 3])
visualize_pcd(pcd_src_colored)